In [1]:
from biomedclip import BiomedCLIP

ModuleNotFoundError: No module named 'models'

In [ ]:
backbone = BiomedCLIP()

In [4]:
backbone.image_processor

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    <function _convert_to_rgb at 0x7fe86faae9e0>
    ToTensor()
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
)

In [5]:
from transformers import CLIPProcessor, CLIPModel

processor = CLIPProcessor.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32")

processor

CLIPProcessor:
- image_processor: CLIPImageProcessor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "CLIPImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

- tokenizer: CLIPTokenizerFast(name_or_path='flaviagiammarino/pubmed-clip-vit-base-patch32', vocab_size=49408, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	49406: AddedToken("<|startoftext|>", rstrip=False, lstrip=F

In [40]:
from open_clip import create_model_from_pretrained, get_tokenizer

model, image_processor = create_model_from_pretrained(
    "hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224"
)
tokenizer = get_tokenizer(
    "hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224")

model.eval()

CustomTextCLIP(
  (visual): TimmModel(
    (trunk): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=768

In [41]:
word = "A photo of"
word_tokenized = tokenizer([word], 256)

print(word_tokenized.shape)

x = word_tokenized

attn_mask = (x != model.text.config.pad_token_id).long()


out = model.text.transformer(input_ids=x, attention_mask=attn_mask)
print(out.last_hidden_state.shape)
pooled_out = model.text.pooler(out, attn_mask)

print(pooled_out.shape)

torch.Size([1, 256])
torch.Size([1, 256, 768])
torch.Size([1, 768])


In [42]:
inputs_embeds = model.text.transformer.get_input_embeddings()(x)

out_v2 = model.text.transformer(
    inputs_embeds=inputs_embeds, attention_mask=attn_mask)

out.last_hidden_state == out_v2.last_hidden_state

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])

In [31]:
import torch

output_attentions = model.text.transformer.config.output_attentions
output_hidden_states = model.text.transformer.config.output_hidden_states

return_dict = model.text.transformer.config.use_return_dict

if model.text.transformer.config.is_decoder:
    model.text.transformer.config.use_cache
else:
    use_cache = False

input_shape = x.size()

batch_size, seq_length = input_shape
device = x.device

# past_key_values_length
past_key_values_length = 0


if hasattr(model.text.transformer.embeddings, "token_type_ids"):
    buffered_token_type_ids = model.text.transformer.embeddings.token_type_ids[:, :seq_length]
    buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
    token_type_ids = buffered_token_type_ids_expanded
else:
    token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

embedding_output = model.text.transformer.embeddings(
    input_ids=x,
    position_ids=None,
    token_type_ids=token_type_ids,
    inputs_embeds=None,
    past_key_values_length=past_key_values_length,
)

embedding_output.shape

torch.Size([1, 256, 768])

In [38]:
with torch.no_grad():
    inputs_embeds = model.text.transformer.get_input_embeddings()(x)

print(inputs_embeds.shape)

embedding_output_v2 = model.text.transformer.embeddings(
    input_ids=None,
    position_ids=None,
    token_type_ids=token_type_ids,
    inputs_embeds=inputs_embeds,
    past_key_values_length=past_key_values_length,
)

correct = embedding_output_v2 == embedding_output

correct.sum() / torch.numel(correct)

torch.Size([1, 256, 768])


tensor(0.8195)

In [39]:
model.text.transformer.get_input_embeddings()

Embedding(30522, 768, padding_idx=0)

In [46]:
print(model.text.pooler)

ClsLastHiddenStatePooler()


In [18]:
"" == True

False

In [5]:
for i, _ in enumerate(model.visual.trunk.blocks.children()):
    print(_)

Block(
  (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (attn): Attention(
    (qkv): Linear(in_features=768, out_features=2304, bias=True)
    (q_norm): Identity()
    (k_norm): Identity()
    (attn_drop): Dropout(p=0.0, inplace=False)
    (proj): Linear(in_features=768, out_features=768, bias=True)
    (proj_drop): Dropout(p=0.0, inplace=False)
  )
  (ls1): Identity()
  (drop_path1): Identity()
  (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (mlp): Mlp(
    (fc1): Linear(in_features=768, out_features=3072, bias=True)
    (act): GELU(approximate='none')
    (drop1): Dropout(p=0.0, inplace=False)
    (norm): Identity()
    (fc2): Linear(in_features=3072, out_features=768, bias=True)
    (drop2): Dropout(p=0.0, inplace=False)
  )
  (ls2): Identity()
  (drop_path2): Identity()
)
Block(
  (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (attn): Attention(
    (qkv): Linear(in_features=768, out_features=2304, bias=True)
    (q_norm): Ide

In [11]:
from open_clip import get_model_config
from open_clip.factory import _get_hf_config

model_name = "hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224"
HF_HUB_PREFIX = "hf-hub:"
model_id = model_name[len(HF_HUB_PREFIX):]
config = _get_hf_config(model_id)

model_cfg = model_cfg = config["model_cfg"]
print(model_cfg["vision_cfg"])

{'timm_model_name': 'vit_base_patch16_224', 'timm_model_pretrained': False, 'timm_pool': '', 'timm_proj': 'linear', 'image_size': 224}


In [15]:
import torch

sample_input = torch.rand(1, 3, 224, 224)

sample_out = model.visual.trunk.forward_features(sample_input)

In [16]:
sample_out.shape

torch.Size([1, 197, 768])

In [17]:
model.visual.trunk.global_pool

'token'